https://www.kaggle.com/clair14/lgbm-bayesianoptimization

In [1]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
import random
import os
from sklearn.metrics import log_loss
from bayes_opt import BayesianOptimization
import lightgbm



d = "C:\kaggle_data\credit_card"
lst = os.listdir(d)
print(lst)
train = pd.read_csv(d + '\\' +lst[3])
test = pd.read_csv(d + '\\' +lst[2])
ss = pd.read_csv(d + '\\' +lst[1])
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)


# Married, Civil marriage
train['income_per_size'] = np.log(train['income_total']/train['family_size'])
test['income_per_size'] = np.log(test['income_total']/test['family_size'])
train.loc[(train['family_type']=='Married')|(train['family_type']=='Civil marriage'),'income_per_size']\
= train['income_per_size'] * 2

test.loc[(test['family_type']=='Married')|(test['family_type']=='Civil marriage'),'income_per_size']\
= test['income_per_size'] * 2

def simple_marry(x):
    if x == 'Married' or x =='Civil marriage':
        return '0'
    elif x == 'Separated' or x == 'Widow':
        return '1'
    else:
        return '2'

for df in [train,test]:
    df['family_bins'] = df['family_type'].apply(simple_marry)

# income_total을 로그변환
train['income_total'] = np.log(train['income_total'])
# train = train.drop('income_total',1)
test['income_total'] = np.log(test['income_total'])
# test = test.drop('income_total',1)

# car와 reality를 합친 새로운 칼럼 careality
train['car'] =train['car'].apply(lambda x: int(x=='Y'))
train['reality'] =train['reality'].apply(lambda x: int(x=='Y'))
test['car'] =test['car'].apply(lambda x: int(x=='Y'))
test['reality'] =test['reality'].apply(lambda x: int(x=='Y'))

train['careality'] = train['car'] + train['reality']
train = train.drop(['car', 'reality'],1)

test['careality'] = test['car'] + test['reality']
test = test.drop(['car', 'reality'],1)

train.drop('gender',1,inplace=True)
test.drop('gender',1,inplace=True)

object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)

enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)


## 제곱
for df in [train,test]:
    df['income_per_size'] = df['income_per_size'].apply(lambda x: x**2)
    
c = 'income_per_size'
mean = train[c].mean()
std = train[c].std()

# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std

#####################
c = 'income_total'
k = 2.2
mean = train[c].mean()
std = train[c].std()
idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
train = train.drop(idxs).reset_index(drop=True)

# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std

#####################
c = 'begin_month'
k = 2.2

train[c] = train[c].apply(lambda x: x**2)
test[c] = test[c].apply(lambda x: x**2)

mean = train[c].mean()
std = train[c].std()

idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
# train = train.drop(idxs).reset_index(drop=True)

# 기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std

######################
c = 'DAYS_BIRTH'
k = 2.2
mean = train[c].mean()
std = train[c].std()
idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
train = train.drop(idxs).reset_index(drop=True)

#기존 
train[c] = (train[c]-mean)/std
test[c] = (test[c]-mean)/std



###################
c = 'DAYS_EMPLOYED'
k = 2.2
mean = train[c].mean()
std = train[c].std()
idxs = train.loc[(train[c]>= mean + k*std)|\
                (train[c]<= mean - k*std)].index
train = train.drop(idxs).reset_index(drop=True)



# ran = ['child_num', 'DAYS_BIRTH', 'DAYS_EMPLOYED','family_size', 'begin_month', 'income_per_size']
# from sklearn.preprocessing import QuantileTransformer
# tf = QuantileTransformer(n_quantiles=100,random_state=42, output_distribution='normal')
# tf.fit(train[ran])
# train[ran] = tf.transform(train[ran])
# test[ran] = tf.transform(test[ran])


['.ipynb_checkpoints', 'sample_submission.csv', 'test.csv', 'train.csv', 'Untitled.ipynb']


## 파라미터 튜닝 완료

In [11]:
n = 10
loss = 0
skf = StratifiedKFold(n_splits=n, shuffle=True, random_state=42)
folds=[]
oof = []
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))
random.seed(42)
models={}
params = {
        "boosting": 'gbdt',
        "objective" : "multiclass",
        "metric" : "multi_logloss", 
        "num_classes" : 3,
        "n_estimators": 1000,  
            "max_depth": 9,            
        "bagging_fraction": 0.9, 
        "bagging_freq":2,         
        "feature_fraction": 0.7, 
        'force_col_wise':True,
        "nthreads":44,
        "seed":42}


for fold in range(n):
    print('$$$$$$$$$$$$$$$$$$$$$$$',fold+1,'번째','$$$$$$$$$$$$$$$$$$$$$$$')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    lgtrain = lightgbm.Dataset(X_train, y_train)
    lgeval = lightgbm.Dataset(X_valid,y_valid)
    model = lightgbm.train(params=params,train_set=lgtrain,
                           valid_sets=lgeval,
                           verbose_eval=100,
                          early_stopping_rounds=30)
    
    y_pred = model.predict(X_valid)
#     oof.append(y_pred)
    loss += log_loss(y_valid,y_pred)
    models[fold] = model

$$$$$$$$$$$$$$$$$$$$$$$ 1 번째 $$$$$$$$$$$$$$$$$$$$$$$
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Total Bins 998
[LightGBM] [Info] Number of data points in the train set: 19358, number of used features: 52
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score -2.108690
[LightGBM] [Info] Start training from score -1.441188
[LightGBM] [Info] Start training from score -0.443234
Training until validation scores don't improve for 30 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[100]	valid_0's multi_logloss: 0.748957
[200]	valid_0's multi_logloss: 0.729443
Early stopping, best iteration is:
[265]	valid_0's mu

[LightGBM] [Info] Total Bins 996
[LightGBM] [Info] Number of data points in the train set: 19358, number of used features: 51
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score -2.109116
[LightGBM] [Info] Start training from score -1.440970
[LightGBM] [Info] Start training from score -0.443234
Training until validation scores don't improve for 30 rounds
[100]	valid_0's multi_logloss: 0.739818
[200]	valid_0's multi_logloss: 0.716591
[300]	valid_0's multi_logloss: 0.709502
Early stopping, best iteration is:
[306]	valid_0's multi_logloss: 0.708807
$$$$$$$$$$$$$$$$$$$$$$$ 10 번째 $$$$$$$$$$$$$$$$$$$$$$$
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [

In [12]:
loss

7.151382661732114

In [7]:
ss.iloc[:,1:]=0
for fold in range(n):
    ss.iloc[:,1:] += models[fold].predict(test.values)/n

In [9]:
ss.to_csv('05_22_lgbm.csv',index=False)

In [38]:
# min_child_samples, min_child_weight 활성화
# loss = 7.1728937306252885

# min_child_samples, min_child_weight 비활성화
# loss = 7.144239538357529

# force_col_wise까지 비활성화
loss

7.144239538357529

In [6]:
ss.iloc[:,1:]=0
for fold in range(n):
    ss.iloc[:,1:] += models[fold].predict(test)/n

In [7]:
ss.to_csv('lgbm_tuned.csv',index=False)

In [16]:
import re

In [21]:
p = re.compile('[0]\W[1-9]{4,5}')

In [22]:
for i in p.finditer(a):
    print(i.group())

0.8491
0.82522


In [5]:
for z in a.split('='):
    if z.startswith(' 0.'):
        print(z)

 0.80141
Best weights from best epoch are automatically used!
5 3 3

Early stopping occurred at epoch 214 with best_epoch 
 0.80541
Best weights from best epoch are automatically used!
 50%|██████████████████                  | 5/10 [06:34<06:28, 77.67s/it]
5 3 3
 60%|█████████████████████▌              | 6/10 [07:45<05:02, 75.51s/it]

Early stopping occurred at epoch 182 with best_epoch 
 0.8057
Best weights from best epoch are automatically used!
5 3 3
 70%|█████████████████████████▏          | 7/10 [08:23<03:09, 63.15s/it]

Early stopping occurred at epoch 95 with best_epoch 
 0.8491
Best weights from best epoch are automatically used!
5 3 3

Early stopping occurred at epoch 167 with best_epoch 
 0.82522
Best weights from best epoch are automatically used!
 80%|████████████████████████████▊       | 8/10 [09:28<02:07, 63.93s/it]
5 3 3
 90%|████████████████████████████████▍   | 9/10 [10:28<01:02, 62.54s/it]

Early stopping occurred at epoch 151 with best_epoch 
 0.8208
Best weights fr